목표 : 무료 요금제 사용자가 유료 사용자로 전환할지 예측  
- 관련 메트릭, confusion matrix, 다른 표준 활용

**방법 1**  
유료 사용한 적 있는 사용자 무료 요금제 시절 데이터 랭글링
- 총 학습 시간
- 참가 퀴즈 수, 강의 수, 시험 횟수, qna 여부 등  
 1) 유료, 무료 사용자 구분하여 PCA 적용  
     -> 유료 사용자와의 유사도 측정 통해 threshold 이상은 전환으로 추측  
     -> threshold 기준을 어떻게 설정해야할지 고민 필요

**방법 2**  
세부적으로 데이터 랭글링
1. 유료로 전환하는 이유
    - 무료로 수강하다보니 강의가 괜찮아서
    - 유료 강의에 원하는 강의, 시험 등이 있어서
    - 그 외
2. 경우의 수
    - 가입한 뒤 바로 유료 전환
    - 가입한 뒤 무료 강의를 일정기간 수강한 이후 전환
    - 아직 전환하지 않은 경우


# 1. 기본 EDA

In [1]:
# Data Load
import pandas as pd
from datetime import datetime

course_info = pd.read_csv('./365_database/365_course_info.csv')
ratings = pd.read_csv('./365_database/365_course_ratings.csv')
exam_info = pd.read_csv('./365_database/365_exam_info.csv')
quiz_info = pd.read_csv('./365_database/365_quiz_info.csv')
student_engagement = pd.read_csv('./365_database/365_student_engagement.csv')
student_exams = pd.read_csv('./365_database/365_student_exams.csv')
student_hub_questions = pd.read_csv('./365_database/365_student_hub_questions.csv')
student_info = pd.read_csv('./365_database/365_student_info.csv')
student_learning = pd.read_csv('./365_database/365_student_learning.csv')
student_purchases = pd.read_csv('./365_database/365_student_purchases.csv')
student_quizzes = pd.read_csv('./365_database/365_student_quizzes.csv')

In [4]:
# 구매 경험 있는 고객
paid_student = student_purchases.student_id.unique()

In [8]:
student_info['paid'] = 0
for n, i in student_info.iterrows():
    if i.student_id in paid_student:
        student_info.loc[n, 'paid'] = 1
    else:
        student_info.loc[n, 'paid'] = 0
        
student_info

,student_id,student_country,date_registered,paid
0,258798,IN,2022-01-01,0
1,258799,CO,2022-01-01,0
2,258800,CA,2022-01-01,1
3,258801,IN,2022-01-01,0
4,258802,US,2022-01-01,0
...,...,...,...,...
35225,295511,CM,2022-10-20,0
35226,295512,CA,2022-10-20,0
35227,295513,SA,2022-10-20,0
35228,295514,NL,2022-10-20,0


In [19]:
paid_info = student_info[student_info.paid == 1]
paid_info['date_paid'] = ''
for n, i in paid_info.iterrows():
    paid_info.loc[n, 'date_paid'] = student_purchases[student_purchases.student_id == i.student_id].date_purchased.values[0]

paid_info['date_registered'] = pd.to_datetime(paid_info['date_registered'])
paid_info['date_paid'] = pd.to_datetime(paid_info['date_paid'])
paid_info

/tmp/ipykernel_1020/2841196499.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paid_info['date_paid'] = ''


,student_id,student_country,date_registered,paid,date_paid
2,258800,CA,2022-01-01,1,2022-01-01
5,258803,US,2022-01-01,1,2022-01-01
20,258818,IN,2022-01-01,1,2022-01-03
23,258822,IN,2022-01-01,1,2022-01-09
33,258832,IN,2022-01-01,1,2022-03-05
...,...,...,...,...,...
34985,295256,CA,2022-10-19,1,2022-10-19
35002,295273,TR,2022-10-19,1,2022-10-19
35079,295355,US,2022-10-19,1,2022-10-19
35211,295497,NL,2022-10-20,1,2022-10-20


In [21]:
paid_info['paid_duration'] = paid_info.date_paid - paid_info.date_registered
paid_info

/tmp/ipykernel_1020/668599084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paid_info['paid_duration'] = paid_info.date_paid - paid_info.date_registered


,student_id,student_country,date_registered,paid,date_paid,paid_duration
2,258800,CA,2022-01-01,1,2022-01-01,0 days
5,258803,US,2022-01-01,1,2022-01-01,0 days
20,258818,IN,2022-01-01,1,2022-01-03,2 days
23,258822,IN,2022-01-01,1,2022-01-09,8 days
33,258832,IN,2022-01-01,1,2022-03-05,63 days
...,...,...,...,...,...,...
34985,295256,CA,2022-10-19,1,2022-10-19,0 days
35002,295273,TR,2022-10-19,1,2022-10-19,0 days
35079,295355,US,2022-10-19,1,2022-10-19,0 days
35211,295497,NL,2022-10-20,1,2022-10-20,0 days


In [24]:
# 3가지 경우의 수
import datetime as dt
FreetoPaid = paid_info[paid_info.paid_duration != dt.timedelta(days=0)] ##1
Paid = paid_info[paid_info.paid_duration == dt.timedelta(days=0)] ##2
Free = student_info[student_info.paid == 0] ##3
display(FreetoPaid, Paid, Free)

,student_id,student_country,date_registered,paid,date_paid,paid_duration
20,258818,IN,2022-01-01,1,2022-01-03,2 days
23,258822,IN,2022-01-01,1,2022-01-09,8 days
33,258832,IN,2022-01-01,1,2022-03-05,63 days
112,258913,US,2022-01-02,1,2022-01-27,25 days
120,258922,US,2022-01-02,1,2022-01-04,2 days
...,...,...,...,...,...,...
34644,294907,FR,2022-10-16,1,2022-10-17,1 days
34673,294936,IN,2022-10-16,1,2022-10-17,1 days
34686,294949,US,2022-10-16,1,2022-10-17,1 days
34822,295088,NL,2022-10-17,1,2022-10-18,1 days


,student_id,student_country,date_registered,paid,date_paid,paid_duration
2,258800,CA,2022-01-01,1,2022-01-01,0 days
5,258803,US,2022-01-01,1,2022-01-01,0 days
62,258862,NL,2022-01-01,1,2022-01-01,0 days
64,258865,CA,2022-01-01,1,2022-01-01,0 days
77,258878,CA,2022-01-01,1,2022-01-01,0 days
...,...,...,...,...,...,...
34985,295256,CA,2022-10-19,1,2022-10-19,0 days
35002,295273,TR,2022-10-19,1,2022-10-19,0 days
35079,295355,US,2022-10-19,1,2022-10-19,0 days
35211,295497,NL,2022-10-20,1,2022-10-20,0 days


,student_id,student_country,date_registered,paid
0,258798,IN,2022-01-01,0
1,258799,CO,2022-01-01,0
3,258801,IN,2022-01-01,0
4,258802,US,2022-01-01,0
6,258804,BD,2022-01-01,0
...,...,...,...,...
35225,295511,CM,2022-10-20,0
35226,295512,CA,2022-10-20,0
35227,295513,SA,2022-10-20,0
35228,295514,NL,2022-10-20,0
